In [1]:
from pathlib import Path

from speechcolab.datasets.gigaspeech import GigaSpeech

from lhotse.recipes.gigaspeech import prepare_gigaspeech

# Select data parts

In [2]:
dataset_parts = ('{XS}', '{TEST}')

# Settings for paths

In [3]:
root_dir = Path('data')
corpus_dir = root_dir / 'GigaSpeech'
output_dir = root_dir / 'gigaspeech_nb'

# Select data parts

In [4]:
dataset_parts = ('{XS}', '{TEST}')

# Download the data

In [5]:
gigaspeech = GigaSpeech(corpus_dir)

In [6]:
for part in dataset_parts:
    # TODO: remove this try-except block in the stable version
    try:
        gigaspeech.download(part)
    except NotImplementedError:
        assert gigaspeech.json_path.is_file()

# Prepare audio and supervision manifests

In [7]:
gigaspeech_manifests_train = prepare_gigaspeech(gigaspeech, dataset_parts, output_dir)

In [8]:
gigaspeech_manifests_train

{'{XS}': {'recordings': RecordingSet(len=79),
  'supervisions': SupervisionSet(len=15304)},
 '{TEST}': {'recordings': RecordingSet(len=131),
  'supervisions': SupervisionSet(len=25619)}}